In [1]:
import numpy as np
import pandas as pd
import os
import json
from collections import Counter

In [20]:
path = "/opt/ml/code/ensemble/"
file_list = os.listdir(path)

In [21]:
retrieval_score = {
    "tfidf": 0.7562,
    "elastic": 0.8576,
    "bm25": 0.8630
}
reader_score = {
    "robertaL": 72.50,
    "robertaB": 66.66,
    "koelectra": 51.66,
    "xlmrobertaL": 56.17,
    "bertB": 58.75,
    "custom1": 71.66,
    "custom2": 72.50,
    "custom3": 71.66,
}

In [22]:
files = []
scores = []
for name in file_list:
    if '.json' in name:
        bef, _ = name.split(".")
        reader, retrieval = bef.split("_")
        scores.append(reader_score[reader]*retrieval_score[retrieval])
        with open(path+name, "r", encoding="utf-8") as f:
            prediction_json = json.load(f)
            files.append(prediction_json)

In [ ]:
(file_list, scores)

In [23]:
final_prediction = dict()

In [ ]:
z = 0
for q_id in files[0]:
    tmp_list = []
    score_dict = dict()
    for i in range(len(files)):
        file = files[i]
        tmp_list.append(file[q_id])
        if file[q_id] not in score_dict:
            score_dict[file[q_id]] = scores[i]
        else:
            score_dict[file[q_id]] += scores[i]
    count = Counter(tmp_list)
    print(count)
    answer = ''
    cnt = 0
    for tmp_answer in count:
        if cnt < count[tmp_answer]:
            cnt = count[tmp_answer]
            answer = tmp_answer
        elif cnt == count[tmp_answer]:
            if score_dict[answer] < score_dict[tmp_answer]:
                answer = tmp_answer
    final_prediction[q_id] = answer

In [25]:
with open(path+"ensemble_output.json", 'w', encoding='utf-8') as make_file:
    json.dump(final_prediction, make_file, indent="\t", ensure_ascii=False)